In [1]:
!python3 -m pip install --upgrade pip

In [2]:
!pip3 install arxiv==2.1.0
!pip3 install python-dotenv tiktoken
# !pip install openai==0.27.8
# !pip install openai==1.2.3
!pip install openai==1.3.4
!pip install -U duckduckgo-search==4.4

In [3]:
import os
import json
import datetime as dt
# from datetime import datetime
import yaml

import warnings


import arxiv
import openai
from openai import OpenAI
from dotenv import load_dotenv

from duckduckgo_search import DDGS, AsyncDDGS
import asyncio

# すべての警告を無視する
warnings.filterwarnings('ignore')

In [4]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [5]:
load_dotenv()

True

In [6]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [7]:
MODEL_NAME = "gpt-3.5-turbo-0125"
# MODEL_NAME = "gpt-3.5-turbo-instruct"
# MODEL_NAME = "gpt-4-0125-preview"
TEMPERATURE = 0.7
# OpenAIクライアントの初期化
client = OpenAI()

In [8]:
functions = \
[
    {
        # 【日付範囲の特定】
        # 関数名
        "name": "date_range_from_word_updated_fixed",
        # 関数の説明
        "description": "与えられた単語から日付の範囲を特定し、文字列で返す。",
        # 関数の引数の定義
        "parameters":
         {
            "type": "object",
            "properties":
             {
                "word":
                {
                    "type": "string",
                    "description": "日付範囲を示唆する単語や文章。例えば、「最近」、「今年」、「今週」、「来週」、「昨日」、または「未来」といったキーワードが含まれる文章が入ります。"
                }
            },
            "required": ["word"],
        }
    },
]


In [9]:
def date_range_from_word_updated_fixed(word):
    # 現在の日時を取得
    now = dt.datetime.now()
    # 日付の範囲を格納する変数を初期化
    date_range = ""
    
    if "最近" in word:
        six_months_ago = now - dt.timedelta(days=182) # 大体半年前
        date_range = f"{six_months_ago.strftime('%Y-%m-%d')}から{now.strftime('%Y-%m-%d')}まで"
    
    elif word == "今年":
        start_of_year = dt.datetime(now.year, 1, 1)
        end_of_year = dt.datetime(now.year, 12, 31)
        date_range = f"{start_of_year.strftime('%Y-%m-%d')}から{end_of_year.strftime('%Y-%m-%d')}まで"
    
    elif word == "今週":
        start_of_week = now - dt.timedelta(days=now.weekday())
        end_of_week = start_of_week + dt.timedelta(days=6)
        date_range = f"{start_of_week.strftime('%Y-%m-%d')}から{end_of_week.strftime('%Y-%m-%d')}まで"
    
    elif word == "来週":
        start_of_next_week = now + dt.timedelta(days=(7 - now.weekday()))
        end_of_next_week = start_of_next_week + dt.timedelta(days=6)
        date_range = f"{start_of_next_week.strftime('%Y-%m-%d')}から{end_of_next_week.strftime('%Y-%m-%d')}まで"
    
    elif word == "昨日":
        yesterday = now - dt.timedelta(days=1)
        date_range = yesterday.strftime('%Y-%m-%d')
    
    elif "未来" in word:
        future_time = word.replace("未来", "").strip()
        if future_time == "":
            start_of_next_year = dt.datetime(now.year + 1, 1, 1)
            end_of_next_year = dt.datetime(now.year + 1, 12, 31)
            date_range = f"{now.strftime('%Y-%m-%d')}から{start_of_next_year.strftime('%Y-%m-%d')}まで"
        elif future_time.isdigit():
            future_date = now + dt.timedelta(days=365 * int(future_time))
            date_range = future_date.strftime('%Y-%m-%d')
        else:
            # 不正確な入力に対する処理を追加するか、エラーメッセージを表示する
            return "不正確な入力: " + word
    
    elif "最新" in word:
        # 「最新の」処理
        six_months_ago = now - dt.timedelta(days=182) # 大体半年前
        date_range = f"{six_months_ago.strftime('%Y-%m-%d')}から{now.strftime('%Y-%m-%d')}までの{word.replace('最新', '')}"
    
    elif "今年" in word:
        # 「今年の」処理
        start_of_year = dt.datetime(now.year, 1, 1)
        end_of_year = dt.datetime(now.year, 12, 31)
        date_range = f"{start_of_year.strftime('%Y-%m-%d')}から{end_of_year.strftime('%Y-%m-%d')}までの{word.replace('今年', '')}"
    
    else:
        # 日付のわかるワードがない場合はそのまま返す
        return word
    
    return date_range


In [10]:
# テスト
print(date_range_from_word_updated_fixed("最近"))
print(date_range_from_word_updated_fixed("今年"))
print(date_range_from_word_updated_fixed("今週"))
print(date_range_from_word_updated_fixed("来週"))
print(date_range_from_word_updated_fixed("昨日"))
print(date_range_from_word_updated_fixed("未来"))
print(date_range_from_word_updated_fixed("未来50年後"))
print(date_range_from_word_updated_fixed("未来5日後"))
print(date_range_from_word_updated_fixed("最新のAIトレンド"))
print(date_range_from_word_updated_fixed("今年のファッショントレンド"))

2023-08-14から2024-02-12まで
2024-01-01から2024-12-31まで
2024-02-12から2024-02-18まで
2024-02-19から2024-02-25まで
2024-02-11
2024-02-12から2025-01-01まで
不正確な入力: 未来50年後
不正確な入力: 未来5日後
2023-08-14から2024-02-12までののAIトレンド
2024-01-01から2024-12-31までののファッショントレンド


In [11]:
query = "2024年, 最新テクノロジートレンド, 注目, 社会, 予想, 影響"
query_list = query.split(",")
query_list

['2024年', ' 最新テクノロジートレンド', ' 注目', ' 社会', ' 予想', ' 影響']

In [12]:
query_re_list = []
for q in query_list:
    
    query_re_list.append(date_range_from_word_updated_fixed(q))
    
query_re_list

['2024年', '2023-08-14から2024-02-12までの テクノロジートレンド', ' 注目', ' 社会', ' 予想', ' 影響']

In [13]:
# query_listの要素をカンマとスペースで結合して元のqueryに戻す
query_reconstructed = ", ".join(query_re_list).strip()
query_reconstructed


'2024年, 2023-08-14から2024-02-12までの テクノロジートレンド,  注目,  社会,  予想,  影響'

In [14]:
# 関数を選択して実行するデモ関数
def function_calling_query(prompt):

    # GPTにプロンプトと関数定義リストを一緒に投入し、Function callingの使用を指示
    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=TEMPERATURE,
        messages=
    	[
    		{ "role": "user", "content": prompt}  # プロンプトを投入
    	],
        functions=functions, # プロンプトと一緒に関数定義リストを投入
        function_call="auto", # Function callingを使用するが、その際、関数の選択はGPTに任せる
    )

    # GPTからの応答を抽出
    message = response.choices[0].message
    print("message: ", message)
    function_call = message.function_call
    print("function_call: ", function_call)
    if function_call is not None:
        # 実行すべき関数名
        function_name = function_call.name
        # その関数に渡すべき引数
        # arguments = json.loads(function_call.arguments)
        # promptを直接引数として使用
        arguments = {"word": prompt}
    
        # 関数の選択と実行
        if function_name == "date_range_from_word_updated_fixed":
            # date_range_from_word_updated_fixed関数を実行
            function_response = date_range_from_word_updated_fixed(arguments["word"])
            answer = function_response
        else:
            # 他の関数が選択された場合の処理
            # この例では、選択された関数がdate_range_from_word_updated_fixedのみであるため、他の分岐は省略
            # answer = "該当する関数がありません。"
            answer = prompt
    else:
        answer = prompt
    return answer

In [15]:
# テスト用プロンプト
prompt = "最近"
# 関数の実行と結果の表示
print(function_calling_query(prompt))

message:  ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"word":"最近"}', name='date_range_from_word_updated_fixed'), tool_calls=None)
function_call:  FunctionCall(arguments='{"word":"最近"}', name='date_range_from_word_updated_fixed')
2023-08-14から2024-02-12まで


In [17]:
query_re_list = []
for q in query_list:
    f_c_q = function_calling_query(q)
    query_re_list.append(f_c_q)
    
query_re_list

message:  ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"word":"2024年"}', name='date_range_from_word_updated_fixed'), tool_calls=None)
function_call:  FunctionCall(arguments='{"word":"2024年"}', name='date_range_from_word_updated_fixed')
message:  ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"word":"最新"}', name='date_range_from_word_updated_fixed'), tool_calls=None)
function_call:  FunctionCall(arguments='{"word":"最新"}', name='date_range_from_word_updated_fixed')
message:  ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"word":"注目"}', name='date_range_from_word_updated_fixed'), tool_calls=None)
function_call:  FunctionCall(arguments='{"word":"注目"}', name='date_range_from_word_updated_fixed')
message:  ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"word":"社会"}', name='date_range_from_word_updated_fixed')

['2024年', '2023-08-14から2024-02-12までの テクノロジートレンド', ' 注目', ' 社会', ' 予想', ' 影響']